In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pylab
import os
import tensorflow as tf
from skimage import io#,transform
import cv2


def convolve(img,fill):
    rgb_r = img[:,:,0]
    rgb_g = img[:,:,1]
    rgb_b = img[:,:,2]
    
    pro_rgb_r = pre_convolve(rgb_r,fill)
    pro_rgb_g = pre_convolve(rgb_g,fill)
    pro_rgb_b = pre_convolve(rgb_b,fill)

    img_pro = np.dstack((pro_rgb_r,pro_rgb_g,pro_rgb_b))
    return img_pro


def pre_convolve(img,fill):
    fill_height = fill.shape[0]
    fill_width = fill.shape[1]

    img_height = img.shape[0]
    img_width = img.shape[1]

    img_pro_height = img_height - fill_height + 1
    img_pro_width = img_width - fill_width + 1

    pro_rgb = np.zeros((img_pro_height,img_pro_width),dtype='uint8')

    for i in range(img_pro_height):
        for j in range(img_pro_width):
            pro_rgb[i][j] = pro_statistic(img[i:i + fill_height,j:j + fill_width],fill)
    return pro_rgb


def pro_statistic(img,fill):
    res = (img * fill).sum()
    if(res > 255):
        res = 255
    elif(res < 0):
        res = abs(res)  # 让负边缘也显现出来
    return res


sobel_y = np.array([[-1,0,1],
                 [-2,0,2],
                 [-1,0,1]])  # 索贝尔算子 边缘检测

sobel_x = np.array([[-1,-2,-1],
                [0,0,0],
                [1,2,1]])

# def merge_image(img1,img2):
#     b1, g1 ,r1 =cv2.split(img1)                 #顺序是b,g,r，不是r,g,b
#     b2, g2 ,r2 =cv2.split(img2)   
#     #img1的Y、U、V
#     Y1 = 0.299*r1 + 0.587*g1 + 0.114*b1
#     U1 = - 0.1687*r1 - 0.3313*g1 + 0.5*b1 + 128
#     V1 = 0.5*r1 - 0.4187*g1 - 0.0813*b1 + 128
#     #img2的Y、U、V
#     Y2 = 0.299*r2 + 0.587*g2 + 0.114*b2
#     U2 = - 0.1687*r2 - 0.3313*g2 + 0.5*b2 + 128
#     V2 = 0.5*r2 - 0.4187*g2 - 0.0813*b2 + 128
#     Y,U,V = Y1+Y2,U1+U2,V1+V2
#     #合并后的r、g、b
#     r_merge = Y + 1.402*(V-128)
#     g_merge = Y - 0.34414*(U-128) - 0.71414*(V-128)
#     b_merge = Y + 1.772*(U-128)
#     img_merge = cv2.merge([b_merge,g_merge,r_merge])
#     return img_merge
# #     img1 = cv2.merge([Y1,U1,V1])
# #     img2 = cv2.merge([Y2,U2,V2])
# #     return img1,img2

# def add_alpha(img):
#     b,g,r,alpha = cv2.split(img)
#     my_alpha = np.ones_like(b) * 255//2
#     new_img = cv2.merge((b,g,r,my_alpha))
#     return new_img

def read_directory(directory_name,save_path):
    for filename in os.listdir(directory_name):
        #print(filename)  # 仅仅是为了测试
#         img = cv2.imread(r'pic\terminal.png')
#         cv2.imwrite(r'pic\MyPic.jpg', img)
        img = cv2.imread(directory_name + "/" + filename)
        #添加alpha通道
        b,g,r = cv2.split(img)
        alpha = np.ones(b.shape,dtype=b.dtype)*255
        alpha[:,:int(b.shape[0])] = 255/2.0
        img_bgra = cv2.merge((b,g,r,alpha))
        
        cv2.imwrite(save_path + "/" + filename+".png",img_bgra)
        img = cv2.imread(save_path + "/" + filename+".png",flags=-1)
        
#         cv2.imwrite(directory_name + "/" + filename +".png", img)
#         img = mpimg.imread(directory_name + "/" + filename +".png")
        #print(img.shape)
        #img.resize((224,224,3))
#         plt.imshow(img)
#         plt.show()
        res1 = convolve(img,sobel_x)
        res2 = convolve(img,sobel_y)
        #添加alpha通道并合并
        b1,g1,r1 = cv2.split(res1)
        alpha1 = np.ones(b1.shape,dtype=b1.dtype)*255
        alpha1[:,:int(b1.shape[0])] = 255/2.0
        res1 = cv2.merge((b1,g1,r1,alpha1))
        
        b2,g2,r2 = cv2.split(res2)
        alpha2 = np.ones(b2.shape,dtype=b2.dtype)*255
        alpha2[:,:int(b2.shape[0])] = 255/2.0
        res2 = cv2.merge((b2,g2,r2,alpha2))
        #res = res1 + res2
        #test = merge_image(res1,res2)
#         res1= cv2.cvtColor(res1, cv2.COLOR_RGB2YUV) #RGB转换为YUV
#         res2= cv2.cvtColor(res2, cv2.COLOR_RGB2YUV) #RGB转换为YUV
        res = res1+res2
#         print(res1.shape)
#         print(res2.shape)
#     plt.imshow(res1)
#     #plt.savefig('C:/Users/kexinlan/Desktop/test/gaosizapsheng/', dpi=750, bbox_inches = 'tight')
#     #plt.savefig(os.path.join("C:/Users/kexinlan/Desktop/test/gaosizapsheng"  , 'exam1.jpg'))
#     plt.show()
#     #cv2.imwrite("C:/Users/kexinlan/Desktop/test/gaosizapsheng"  +  filename, res1)

#     plt.imshow(res2)
#    # plt.savefig('C:/Users/kexinlan/Desktop/test/gaosizapsheng/test1.jpg', dpi=750, bbox_inches = 'tight')
#     #plt.savefig(os.path.join("C:/Users/kexinlan/Desktop/test/gaosizapsheng"  , 'exam2.jpg'))
#     plt.show()
#     #cv2.imwrite("C:/Users/kexinlan/Desktop/test/gaosizapsheng"  +  filename,res2)

#     plt.imshow(res)
#     #plt.savefig(os.path.join("C:/Users/kexinlan/Desktop/test/gaosizapsheng"  , 'exam3.jpg'))
#     plt.show()
#     #cv2.imwrite("C:/Users/kexinlan/Desktop/test/gaosizapsheng"  +  filename, res)
    
        #test = transform.resize(test, (224, 224))
#     plt.imshow(img)
#     plt.show()
#     print(img.shape)/test/
#         print(img.shape)
#         print(res.shape)
        #img.resize((img.shape[0]-1,img.shape[1]-1,3))
        res = cv2.resize(res, (img.shape[1],img.shape[0]),0,0, cv2.INTER_LINEAR)  # 修改尺寸
        #res.resize((img.shape[0],img.shape[1],4),Image.BILINEAR)
#     plt.imshow(res)
#     plt.show()
#         print(res1.shape)
#         print(res2.shape)
#         print(res.shape)
#         test = tf.concat([img,res],-1)
#         test = tf.cast(test, dtype=tf.uint8)# 这一步转换张量数据类型很重要
#         test = tf.image.encode_png(res)# 编码回图片
        #test = img*0.5+res*0.5
        #图像叠加
        #test = cv2.add(img, res)
        #权重叠加
        #test = cv2.addWeighted(img, 0.7, res, 0.3,0)
#         b_res,g_res,r_res,alpha_res = cv2.split(res)
#         test_alpha = np.ones(b.shape,dtype=b.dtype)*255
#         test_alpha[:,:int(b.shape[0])] = alpha/2.0 + alpha_res/2.0
#         test = cv2.merge((b,g,r,test_alpha))
        #cv2.imshow('image',img)
        #cv2.imshow("test", test)
        #cv2.imwrite("/Users/jiangshihua/Desktop/1/training/training/e0/test/new_"+filename,test)
        io.imsave(save_path+"/"+filename+".png",res)  #循环保存图片
#         plt.imshow(test)
#         plt.show()



In [4]:
load_path = "/Users/jiangshihua/Desktop/Created Dataset1/training/e0"
save_path = "/Users/jiangshihua/Desktop/Created Dataset1/training/new_e0"
a = read_directory(load_path,save_path)#这里传入所要读取文件夹的绝对路径，加引号（引号不能省略！）

In [5]:
load_path = "/Users/jiangshihua/Desktop/Created Dataset1/training/e1"
save_path = "/Users/jiangshihua/Desktop/Created Dataset1/training/new_e1"
a = read_directory(load_path,save_path)#这里传入所要读取文件夹的绝对路径，加引号（引号不能省略！）

In [6]:
load_path = "/Users/jiangshihua/Desktop/Created Dataset1/training/e2"
save_path = "/Users/jiangshihua/Desktop/Created Dataset1/training/new_e2"
a = read_directory(load_path,save_path)#这里传入所要读取文件夹的绝对路径，加引号（引号不能省略！）